In [9]:
import os
import re
import pandas as pd
from tqdm import tqdm
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (
    adjusted_rand_score,
    adjusted_mutual_info_score,
    silhouette_score,
    homogeneity_score,
    completeness_score,
    v_measure_score,
)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Путь к объединенным CSV-файлам
combined_path = "../combined_csvs"
combined_files = [f for f in os.listdir(combined_path) if f.endswith(".csv")]

# Сортировка файлов по числу пакетов (например, _250_)
combined_files.sort(key=lambda f: int(re.search(r"(\d+)_packets", f).group(1)))

# Создание папки для промежуточных файлов
intermediate_path = "./unsupervised_models_results/"
os.makedirs(intermediate_path, exist_ok=True)

# Итоговый список для всех результатов
all_results = []

# Проход по каждому CSV-файлу
for combined_file in tqdm(combined_files, desc="Обработка файлов"):
    df = pd.read_csv(os.path.join(combined_path, combined_file))
    X = df.drop(columns=["Label"])
    y_true = df["Label"]

    # Извлечение числа уникальных классов и размера пакетов из имени файла
    n_classes = y_true.nunique()
    packet_size = int(re.search(r"(\d+)_packets", combined_file).group(1))

    print(f"Обучение на выборках в {packet_size} пакетов")

    # Нормализация данных
    try:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    except ValueError as e:
        print(f"  ! Ошибка при нормализации данных для файла '{combined_file}': {e}")
        continue

    connectivity = kneighbors_graph(X_scaled, n_neighbors=10)

    # Словарь моделей с учетом числа классов
    models = {
        # "K-Means": KMeans(n_clusters=n_classes, random_state=42),
        # "DBSCAN": DBSCAN(eps=0.5, min_samples=10),
        "Agglomerative Clustering": AgglomerativeClustering(
            n_clusters=n_classes, connectivity=connectivity
        ),
        # "Gaussian Mixture": GaussianMixture(n_components=n_classes, random_state=42),
        # "Spectral Clustering": SpectralClustering(n_clusters=n_classes, random_state=42),
    }

    # Проход по каждой модели
    for model_name, model in models.items():
        results = {
            "Model": model_name,
            "File": combined_file,
            "Packet Size": packet_size,
        }

        print(f"  - Обучение модели {model_name}")
        try:
            # Обучение модели и предсказания
            if hasattr(model, "fit_predict"):
                y_pred = model.fit_predict(X_scaled)
            else:
                model.fit(X_scaled)
                y_pred = model.predict(X_scaled)

            # Расчёт метрик
            results["Adjusted Rand Index"] = adjusted_rand_score(y_true, y_pred)
            results["Adjusted Mutual Information"] = adjusted_mutual_info_score(
                y_true, y_pred
            )
            results["Silhouette Score"] = silhouette_score(X_scaled, y_pred)
            results["Homogeneity"] = homogeneity_score(y_true, y_pred)
            results["Completeness"] = completeness_score(y_true, y_pred)
            results["V-Measure"] = v_measure_score(y_true, y_pred)

        except ValueError as e:
            # Пропуск метрик при ошибке
            results.update(
                {
                    "Adjusted Rand Index": "0.0",
                    "Adjusted Mutual Information": "0.0",
                    "Silhouette Score": "0.0",
                    "Homogeneity": "0.0",
                    "Completeness": "0.0",
                    "V-Measure": "0.0",
                }
            )
            print(
                f"  ! Ошибка обучения модели '{model_name}' для файла '{combined_file}': {e}"
            )

        # Добавление результата
        all_results.append(results)

        # Промежуточный CSV для модели
        model_results_df = pd.DataFrame([results])
        model_results_file = os.path.join(
            intermediate_path, f"{model_name}_unsupervised_results.csv"
        )

        # Если файл существует, добавляем данные
        if os.path.exists(model_results_file):
            model_results_df.to_csv(
                model_results_file,
                mode="a",
                index=False,
                header=False,
                float_format="%.3f",
            )
        else:
            model_results_df.to_csv(
                model_results_file, index=False, float_format="%.3f"
            )

# Итоговый файл с результатами всех моделей
final_results_df = pd.DataFrame(all_results)
final_results_df.to_csv(
    "final_unsupervised_models_results.csv", index=False, float_format="%.3f"
)
print("Итоговый файл сохранён в 'final_unsupervised_models_results.csv'")

Обработка файлов:   0%|          | 0/9 [00:00<?, ?it/s]

Обучение на выборках в 10 пакетов
  - Обучение модели Agglomerative Clustering


Обработка файлов:  11%|█         | 1/9 [04:31<36:12, 271.60s/it]

Обучение на выборках в 50 пакетов
  - Обучение модели Agglomerative Clustering


Обработка файлов:  22%|██▏       | 2/9 [04:39<13:34, 116.34s/it]

...

Обучение на выборках в 500 пакетов
  - Обучение модели Agglomerative Clustering


Обработка файлов:  67%|██████▋   | 6/9 [04:42<00:49, 16.63s/it]

Обучение на выборках в 750 пакетов
  - Обучение модели Agglomerative Clustering
Обучение на выборках в 1000 пакетов
  - Обучение модели Agglomerative Clustering


Обработка файлов: 100%|██████████| 9/9 [04:42<00:00, 31.42s/it]

Обучение на выборках в 5000 пакетов
  - Обучение модели Agglomerative Clustering
Обучение на выборках в 10000 пакетов
  - Обучение модели Agglomerative Clustering
Итоговый файл сохранён в 'final_unsupervised_models_results.csv'


In [ ]:
# Создание папки для модифицированных промежуточных файлов
intermediate_path_modified = "./unsupervised_models_results_modified/"
os.makedirs(intermediate_path_modified, exist_ok=True)

# Путь к промежуточным файлам
intermediate_path = "./unsupervised_models_results/"
model_results_files = [
    f for f in os.listdir(intermediate_path) if f.endswith("_unsupervised_results.csv")
]

# Создаем итоговый DataFrame для всех моделей
final_results = []

# Процесс обработки каждого промежуточного файла модели
for model_results_file in model_results_files:
    # Чтение промежуточного CSV файла
    df = pd.read_csv(os.path.join(intermediate_path, model_results_file))
    # display(df)

    # Получаем список метрик, исключая "Packet Size" и "Model"
    metrics_columns = [col for col in df.columns if col not in ['Packet Size', 'Model', 'File']]

    # Определение лучшей выборки для каждой метрики
    best = {}
    counts = {}

    # Добавление столбца Counts для каждой выборки
    max1_counts_column = []
    for index, row in df.iterrows():
        max1_count = sum(1 for col in metrics_columns if row[col] == df[col].max())  # Считаем, сколько раз максимальное значение встречается
        max1_counts_column.append(max1_count)

    df["max 1"] = max1_counts_column

    # Для каждой метрики находим наилучшие выборки
    for col in metrics_columns:  # Перебираем только метрики
        max_value = df[col].max()  # Находим максимальное значение для метрики
        best[col] = df[df[col] == max_value]["Packet Size"].tolist()  # Сохраняем размеры выборок с максимальным значением

        # Подсчитаем, сколько раз каждая выборка была максимальной для этой метрики
        counts[col] = df[col].value_counts().get(max_value, 0)  # Подсчитаем сколько раз максимальное значение встречается

    # Строка Best
    best_row = {"Packet Size": "Best"}
    for col in metrics_columns:
        best_row[col] = best[col]  # Присваиваем лучшие выборки для каждой метрики

    # Добавление строки Best в DataFrame
    df_best = pd.DataFrame([best_row])

    # Добавление строки Best в DataFrame
    df = pd.concat([df, df_best], ignore_index=True)
    
    model_results_file_modified = os.path.join(
            intermediate_path_modified, f"{os.path.splitext(model_results_file)[0]}_modified.csv"
        )

    # Запись результатов в новый CSV файл
    df.to_csv(model_results_file_modified, index=False)